In [31]:
import pandas as pd
from pathlib import Path
import re
from rapidfuzz import fuzz, process
from geopy.geocoders import Nominatim
import us
from datetime import datetime

BASE_DIR = Path("..")
ORIGINAL_DATA_DIR = BASE_DIR / "original_data"
CLEAN_DATA_DIR = BASE_DIR / "clean_data"
FILE_PATH = ORIGINAL_DATA_DIR / "nsf_awards_us_2019_2024.csv"
TARGET_PATH = CLEAN_DATA_DIR / "nsf_awards_us_2019_2024.csv"
def describe_dataset(df):
    summary = []
    for col in df.columns:
        col_data = df[col]
        summary.append({
            'Column': col,
            'Type': col_data.dtype,
            'Nulls': col_data.isnull().sum(),
            'Null %': round(col_data.isnull().mean() * 100, 2),
            'Unique': col_data.nunique(dropna=True),
            'Min': col_data.min() if pd.api.types.is_numeric_dtype(col_data) else None,
            'Max': col_data.max() if pd.api.types.is_numeric_dtype(col_data) else None,
            'Mean': round(col_data.mean(), 3) if pd.api.types.is_numeric_dtype(col_data) else None,
            'Std': round(col_data.std(), 3) if pd.api.types.is_numeric_dtype(col_data) else None,
            'Example values': col_data.dropna().unique()[:3].tolist()  # first 3 unique examples
        })
    
    summary_df = pd.DataFrame(summary)
    return summary_df



In [32]:
data = pd.read_csv(FILE_PATH)



In [33]:
describe_dataset(data)

,Column,Type,Nulls,Null %,Unique,Min,Max,Mean,Std,Example values
0,abstractText,object,235,0.39,38292,None,None,None,None,[The emergence of practical applications for q...
1,agency,object,0,0.00,2,None,None,None,None,"[NSF, US]"
2,awardeeCountryCode,object,0,0.00,156,None,None,None,None,"[US, Department of Energy Oak Ridge Operations..."
3,awardeeName,object,0,0.00,4079,None,None,None,None,"[OHIO STATE UNIVERSITY, THE, University of Haw..."
4,awardeeStateCode,object,0,0.00,207,None,None,None,None,"[OH, HI, GA]"
5,date,object,0,0.00,1644,None,None,None,None,"[12/13/2024, 12/02/2024, 12/10/2024]"
6,startDate,object,0,0.00,310,None,None,None,None,"[12/15/2024, 12/01/2024, 11/15/2024]"
7,expDate,object,0,0.00,444,None,None,None,None,"[11/30/2026, 01/31/2027, 08/31/2025]"
8,title,object,17,0.03,38363,None,None,None,None,[NQVL:QSTD:Pilot: Distributed-Entanglement Qua...


In [34]:
import pandas as pd
from datetime import datetime

def normalize_cols(cols):
    fixed = []
    for c in cols:
        c_clean = c.strip()
        if c_clean.lower().endswith("bstracttext"):
            c_clean = "abstractText"
        fixed.append(c_clean)
    return fixed

data.columns = normalize_cols(data.columns)

for col in ["startDate", "expDate", "date"]:
    if col in data.columns:
        parsed = pd.to_datetime(data[col], format="%m/%d/%Y", errors="coerce")
        if parsed.isna().mean() > 0.5:
            parsed = pd.to_datetime(data[col], errors="coerce", infer_datetime_format=True)
        data[col] = parsed

target = datetime(2024, 12, 31)
data = data[
    (data["startDate"].notna()) &
    (data["expDate"].notna()) &
    (data["startDate"] <= target) &
    (data["expDate"] >= target)
].copy()

keep = ["id", "awardeeName", "awardeeStateCode", "startDate", "expDate", "title", "abstractText", "nsf_url"]
keep_existing = [c for c in keep if c in data.columns]
data = data[keep_existing].reset_index(drop=True)

print(f"✅ Active on 2024-12-31: {len(data)} rows; columns kept: {keep_existing}")


✅ Active on 2024-12-31: 38114 rows; columns kept: ['awardeeName', 'awardeeStateCode', 'startDate', 'expDate', 'title', 'abstractText']


In [35]:
string_cols = ["title", "abstractText", "awardeeName","awardeeStateCode"]

for col in string_cols:
    if col in ["title", "abstractText"]:
        def clean_to_string(text):
            if pd.isna(text):
                return ""
            text = text.lower()
            text = re.sub(r"[^a-z\s]", " ", text)
            text = re.sub(r"\s+", " ", text)
            return text.strip()

        data[col] = data[col].astype("string").apply(clean_to_string)
    else:
        data[col] = data[col].astype("string").str.strip()

In [36]:

us_state_abbrev = {
    'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas',
    'CA': 'California', 'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware',
    'FL': 'Florida', 'GA': 'Georgia', 'HI': 'Hawaii', 'ID': 'Idaho',
    'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas',
    'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland',
    'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi',
    'MO': 'Missouri', 'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada',
    'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico', 'NY': 'New York',
    'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio', 'OK': 'Oklahoma',
    'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina',
    'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah',
    'VT': 'Vermont', 'VA': 'Virginia', 'WA': 'Washington', 'WV': 'West Virginia',
    'WI': 'Wisconsin', 'WY': 'Wyoming', 'DC': 'District of Columbia',
    'PR': 'Puerto Rico', 'GU': 'Guam', 'VI': 'Virgin Islands', 'AS': 'American Samoa'
}

data["org_state_full"] = data["awardeeStateCode"].map(us_state_abbrev)

print(f"Unmatched rows: {data[data['org_state_full'].isna()]}")

print(len(us_state_abbrev.values()))

Unmatched rows:                      awardeeName awardeeStateCode  startDate    expDate  \
10699  NORTHERN MARIANAS COLLEGE               MP 2023-10-01 2026-09-30   

                                                   title  \
10699  collaborative research epiic cultivating innov...   

                                            abstractText org_state_full  
10699  this is a collaborative project between the fo...            NaN  
55


In [37]:
data = data.dropna()

In [ ]:
valid_states_and_dc = [
    'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN',
    'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV',
    'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN',
    'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY', 'DC'
]
data = data[data["org_state"].isin(valid_states_and_dc)].copy()


<StringArray>
['OH', 'HI', 'GA', 'CA', 'TX', 'FL', 'CT', 'SC', 'MI', 'NY', 'PA', 'AL', 'MO',
 'TN', 'IL', 'MD', 'UT', 'VA', 'IN', 'MN', 'OR', 'NE', 'WI', 'MA', 'KY', 'NJ',
 'LA', 'MT', 'AZ', 'ME', 'IA', 'NM', 'RI', 'WA', 'NV', 'AR', 'SD', 'NC', 'DC',
 'CO', 'ID', 'KS', 'WV', 'VT', 'DE', 'OK', 'NH', 'WY', 'PR', 'AK', 'MS', 'ND',
 'VI', 'GU']
Length: 54, dtype: string

In [39]:
data.to_csv(TARGET_PATH, index=False)
